<a href="https://colab.research.google.com/github/mickeykim70/udemy_tensorflow/blob/main/artificial_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Colab path work



In [9]:
file_path = './data/Part 1 - Artificial Neural Networks (ANN)/'
try:
    # CSV 파일 읽기
    dataset = pd.read_csv(file_path)
    print("파일을 성공적으로 불러왔습니다!")
except FileNotFoundError:
    print(f"지정된 파일을 찾을 수 없습니다: {file_path}")

NameError: name 'pd' is not defined

# **Artificial Neural Network**

Importing the Libraries

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [11]:
tf.__version__

'2.16.2'

# Part 1 - Data Preprocessing

## Importing dataset

In [16]:
dataset = pd.read_csv('./data/Part 1 - Artificial Neural Networks (ANN)/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values
print(X[:5], y[:5])

[[619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]
 [608 'Spain' 'Female' 41 1 83807.86 1 0 1 112542.58]
 [502 'France' 'Female' 42 8 159660.8 3 1 0 113931.57]
 [699 'France' 'Female' 39 1 0.0 2 0 0 93826.63]
 [850 'Spain' 'Female' 43 2 125510.82 1 1 1 79084.1]] [1 0 1 0 0]


## Encoding Categorical data

### Encoding the **Independent** variable

In [5]:
# Label Encoding for Gender column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2]) # Gender

In [6]:
X[:10]

array([[619, 'France', 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 'Spain', 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 'France', 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [699, 'France', 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [850, 'Spain', 0, 43, 2, 125510.82, 1, 1, 1, 79084.1],
       [645, 'Spain', 1, 44, 8, 113755.78, 2, 1, 0, 149756.71],
       [822, 'France', 1, 50, 7, 0.0, 2, 1, 1, 10062.8],
       [376, 'Germany', 0, 29, 4, 115046.74, 4, 1, 0, 119346.88],
       [501, 'France', 1, 44, 4, 142051.07, 2, 0, 1, 74940.5],
       [684, 'France', 1, 27, 2, 134603.88, 1, 1, 1, 71725.73]],
      dtype=object)

In [7]:
# Label Encoding for Geography column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [8]:
X[:5] # 1,0,0 : france ... 나머지는 뒤로 통과(619, 608...)

array([[1.0, 0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [0.0, 0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [1.0, 0.0, 0.0, 502, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [1.0, 0.0, 0.0, 699, 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [0.0, 0.0, 1.0, 850, 0, 43, 2, 125510.82, 1, 1, 1, 79084.1]],
      dtype=object)

### Split data set : Train / Test

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train[:1,]

array([[0.0, 0.0, 1.0, 667, 0, 34, 5, 0.0, 2, 1, 0, 163830.64]],
      dtype=object)

## Feature Scaling

In [10]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

# 강의 코드는 원핫 인코딩 된 데이터까지 모두 스케일링 하므로 잘못됨
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)


# Gender(4번)을 제외하고 CreditScore(3번)와 Age(5번) 이후의 수치형 변수들만 스케일링
numerical_indices = [3] + list(range(5, 12))  # [3, 5, 6, 7, 8, 9, 10, 11]
X_train[:, numerical_indices] = sc.fit_transform(X_train[:, numerical_indices])
X_test[:, numerical_indices] = sc.transform(X_test[:, numerical_indices])
X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)

In [11]:
X_train[:1,]

array([[ 0.        ,  0.        ,  1.        ,  0.16958176,  0.        ,
        -0.46460795,  0.00666099, -1.2157174 ,  0.8095029 ,  0.642595  ,
        -1.0322704 ,  1.1064316 ]], dtype=float32)

# Part 2 - Building ANN

## Initialize ANN

In [12]:
import keras
from keras import layers, activations, models

In [13]:
keras.__version__

'3.5.0'

In [14]:
ann = models.Sequential()

## Add Layers

In [15]:
# First Input layer
ann.add(layers.Dense(units=6, activation='relu'))
# Second Hidden layer
ann.add(layers.Dense(units=6, activation='relu'))
# Output layer
ann.add(layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training

## Compile ANN

In [16]:
ann.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    )

## Train ANN

In [17]:
ann.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=100
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5919 - loss: 0.6707
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8023 - loss: 0.4773
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8085 - loss: 0.4290
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8064 - loss: 0.4295
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8076 - loss: 0.4170
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8078 - loss: 0.4134
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8118 - loss: 0.4085
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8193 - loss: 0.3958
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8327 - loss: 0.3711
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8345 - loss: 0.3793
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8456 - loss: 0.3609
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━

# Part 4 - Prediction

Case 1.
France - 600 - male - 40 years old - 3 years - 60000 - 2 - yes - yes - 50,000

In [25]:
new_data = np.array([[1,0,0,600,1,40,3,60000,2,1,1,50000]])

# 2. 수치형 변수만 스케일링 (Geography와 Gender 제외)
numerical_indices = [3] + list(range(5, 12))  # [3, 5, 6, 7, 8, 9, 10, 11]
new_data[:, numerical_indices] = sc.transform(new_data[:, numerical_indices])

ann.predict(new_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


array([[0.03363222]], dtype=float32)

## Predicting the Test set results

In [26]:
y_pred = ann.predict(X_test)
y_pred = (y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Confusion Matrix

In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1521   74]
 [ 201  204]]


0.8625